In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer, SFTConfig


model_dir = "./Qwen3-8B"
train_data_path = "train_data_augmented.jsonl"
output_dir = "./qwen3-8b-lora-finetuned-2"

dataset = load_dataset("json", data_files=train_data_path, split="train")

model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)


Generating train split: 0 examples [00:00, ? examples/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
training_args = SFTConfig(
    output_dir="./qwen3_lora_sft",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    logging_steps=10,
    save_steps=100,
    learning_rate=1e-4,
    fp16=True,
    lr_scheduler_type="cosine",
    report_to="swanlab",
)

# 初始化 SFTTrainer（自动应用 LoRA）
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    peft_config=lora_config,
)

/root/miniconda3/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:289: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/root/miniconda3/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:366: UserWarning: You passed a `dataset_kwargs` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/7951 [00:00<?, ? examples/s]

/root/miniconda3/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:413: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [4]:
trainer.train()

swanlab: Tracking run with swanlab version 0.6.3                                   
swanlab: Run data will be saved locally in /root/swanlog/run-20250614_205710-6c031199
swanlab: 👋 Hi magician10001, welcome to swanlab!
swanlab: Syncing run ./qwen3_lora_sft to the cloud
swanlab: 🏠 View project at https://swanlab.cn/@magician10001/root
swanlab: 🚀 View run at https://swanlab.cn/@magician10001/root/runs/nxgupycysl16sg357si9b


Step,Training Loss
10,2.026900
20,0.715000
30,0.688900
40,0.680400
50,0.601200
60,0.635100
70,0.590900
80,0.610300
90,0.592000
100,0.632400


TrainOutput(global_step=994, training_loss=0.5245539703119688, metrics={'train_runtime': 1398.6039, 'train_samples_per_second': 11.37, 'train_steps_per_second': 0.711, 'total_flos': 1.705403974979113e+17, 'train_loss': 0.5245539703119688, 'epoch': 2.0})

In [5]:
final_lora_path = os.path.join(output_dir, "final_checkpoint")
print(f"训练完成，正在保存最终的 LoRA 适配器到 {final_lora_path}...")
trainer.save_model(final_lora_path)
print("适配器保存完毕！")

训练完成，正在保存最终的 LoRA 适配器到 ./qwen3-8b-lora-finetuned-2/final_checkpoint...
适配器保存完毕！


In [1]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# ============ 路径配置 ============ #
base_model_path = "./Qwen3-8B"
lora_model_path = "./qwen3-8b-lora-finetuned-2/final_checkpoint"
test_file_path = "test1.json"
output_file_path = "output_augmented.txt"

# ============ System Prompt 模板 ============ #
system_prompt = (
    "请从文本中抽取仇恨言论四元组，要求：\n"
    "1. 严格按照以下格式回复：(评论对象 | 论点 | 目标群体 | 是否仇恨 [END])，直接输出，不要解释。\n"
    "2. 如有多个四元组，两两之间用[SEP]分隔。\n"
    "3. 目标群体可以包含以下6项中的一项或多项：Region、Racism、Sexism、LGBTQ、others、non-hate。"
    "注意仅当‘是否仇恨’项为‘non-hate’时，‘目标群体’项才为‘non-hate’。"
)

# ============ 加载 tokenizer 和模型 ============ #
print("加载模型中...")
tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=True)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

model = PeftModel.from_pretrained(base_model, lora_model_path)
model.eval()

# ============ 读取测试样本 ============ #
with open(test_file_path, "r", encoding="utf-8") as f:
    test_data = json.load(f)

# ============ 推理函数 ============ #
def extract_final_line(text):
    """提取输出中的最后一行（非空，非标记行）"""
    lines = [line.strip() for line in text.strip().split("\n") if line.strip()]
    return lines[-1] if lines else ""

# ============ 批量推理并写入 ============ #
print("开始推理...")
with open(output_file_path, "w", encoding="utf-8") as out_file:
    for idx, example in enumerate(test_data):
        user_input = example["content"]
        full_prompt = (
            f"<|im_start|>system\n{system_prompt}<|im_end|>\n"
            f"<|im_start|>user\n{user_input}<|im_end|>\n"
            f"<|im_start|>assistant\n"
        )

        inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=128,
                do_sample=True,
                temperature=0.1,
                top_p=0.8,
                repetition_penalty=1.1,
                eos_token_id=tokenizer.eos_token_id
            )

        response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
        final_line = extract_final_line(response)
        out_file.write(final_line + "\n")
        print(f"[{idx+1}/{len(test_data)}] 完成")

print(f"全部推理完成，结果写入：{output_file_path}")


加载模型中...


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

开始推理...
[1/2000] 完成
[2/2000] 完成
[3/2000] 完成
[4/2000] 完成
[5/2000] 完成
[6/2000] 完成
[7/2000] 完成
[8/2000] 完成
[9/2000] 完成
[10/2000] 完成
[11/2000] 完成
[12/2000] 完成
[13/2000] 完成
[14/2000] 完成
[15/2000] 完成
[16/2000] 完成
[17/2000] 完成
[18/2000] 完成
[19/2000] 完成
[20/2000] 完成
[21/2000] 完成
[22/2000] 完成
[23/2000] 完成
[24/2000] 完成
[25/2000] 完成
[26/2000] 完成
[27/2000] 完成
[28/2000] 完成
[29/2000] 完成
[30/2000] 完成
[31/2000] 完成
[32/2000] 完成
[33/2000] 完成
[34/2000] 完成
[35/2000] 完成
[36/2000] 完成
[37/2000] 完成
[38/2000] 完成
[39/2000] 完成
[40/2000] 完成
[41/2000] 完成
[42/2000] 完成
[43/2000] 完成
[44/2000] 完成
[45/2000] 完成
[46/2000] 完成
[47/2000] 完成
[48/2000] 完成
[49/2000] 完成
[50/2000] 完成
[51/2000] 完成
[52/2000] 完成
[53/2000] 完成
[54/2000] 完成
[55/2000] 完成
[56/2000] 完成
[57/2000] 完成
[58/2000] 完成
[59/2000] 完成
[60/2000] 完成
[61/2000] 完成
[62/2000] 完成
[63/2000] 完成
[64/2000] 完成
[65/2000] 完成
[66/2000] 完成
[67/2000] 完成
[68/2000] 完成
[69/2000] 完成
[70/2000] 完成
[71/2000] 完成
[72/2000] 完成
[73/2000] 完成
[74/2000] 完成
[75/2000] 完成
[76/2000] 完成
[77/2000] 完成
